In [496]:
# Specifying all the imports
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from dotenv import load_dotenv
from langchain.output_parsers import ResponseSchema, StructuredOutputParser, ListOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts.few_shot import FewShotPromptTemplate
import re
import os

In [497]:
# Loading dotenv and specifying tools
load_dotenv()

def get_numbers(query: str):
    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)
    template = """
    You are smart agent that can idenify all the client ids to solve the query. 
    If no start point for a client is mentioned, assume client 1 is the first client.
    id. From the query, return all the ids that the query refers to as a list of integers

    {format_instructions}
    {query}
    """


    class Numbers(BaseModel):
        numbers: str = Field(description="should say 'the numbers extracted from the query'")
        relevant_numbers: list = Field(description="should be a list of all the relevant numbers the llm has picked up")

    parser = JsonOutputParser(pydantic_object=Numbers)

    prompt = PromptTemplate(
        input_variables = ['query'],
        template=template,
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )


    chain = prompt | llm | parser

    ans = chain.invoke(query)
    return ans['relevant_numbers']
        

@tool
def search_tool(query : str):
    """ 
    Takes a user query and performs an online search to provide
    an answer to the query
    """
    
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def create_embeddings(query : str):
    """
    Uses the content from the file path to create embeddings on the inputted data. Using the query
    provided, perform a similarity search on the query and return the output. If you can not provide a response,
    say I don't know rather than providing an incorrect response.
    """

    #file_path = r'C:\GenAI\Final Project\GenAIGroupProject\apple.txt'
    file_path = r'./apple.txt'
    file_path = file_path.strip()
    loader = TextLoader(file_path, encoding= 'utf-8')
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunk = text_splitter.split_documents(documents=document)
    embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
    vectorstore = FAISS.from_documents(chunk, embeddings)

    docs = vectorstore.similarity_search(query)
    return docs[0].page_content


@tool
def query_database(query : str):
    """
    This function queries the SQL database based on the query that the user provides
    to provide insight into the current client stock portfolio
    """

    # Initialize the JSON output key parser


    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)
    db = SQLDatabase.from_uri('sqlite:///client_db')
    agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
    result = agent_executor.invoke(query)

    # def extract_clients(query: str) -> list:
    #     """Extract a list of client ids based on the input query."""
    #     client_numbers = re.findall(r'client (\d+)', query)
    #     clients = [f"client{i}" for i in client_numbers]
    #     #print(f"my clients are {clients}")
    #     return clients
    
    # def extract_clients(query: str) -> list:
    #     res = [int(i) for i in query.split() if i.isdigit()]
    #     return res

    # clients = extract_clients(query)

    #clients = [f"client{i}" for i in range(1, query.count("client") + 1)]

    # Define the response schemas dynamically based on the number of clients
    client = get_numbers(query)
    for i in client:
        response_schemas = [
        ResponseSchema(name=f'client {i}', description=f"A dictionary containing the stock tickers and their percentage allocations for client {i}.")
        ]
    
    # Initialize the structured output parser
    # output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    # # Get the format instructions
    # format_instructions = output_parser.get_format_instructions()

    # # Define the prompt template
    # prompt_template = PromptTemplate(
    #     template="Create a new key, value pair for every new client mentioned in the question.\n{format_instructions}\nQuestion: {question}",
    #     input_variables=["question"],
    #     partial_variables={"format_instructions": format_instructions},
    # )

    
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


    format_instructions = output_parser.get_format_instructions()
    
    prompt = PromptTemplate(
    template="create a new key, value pair for every new client mentioned in the question.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

    chain = prompt | llm | output_parser

    ans = chain.invoke({"question": result})

    if not isinstance(ans, dict):
        ans = {"numbers": "the numbers extracted from the query", "relevant_numbers": ans}
    
    return ans

    

    
@tool
def portfolio_allocation(query : str):
    """
    Based on the allocation stratergy provided as a query, change the allocation in the
    client stock portfolio in the SQL database
    """
    #query = Reallocte the portfolio for client 1 with a balanced strategy

    #  1. Read in the client database using the query database tool

    template = """
    You are a helpful assistant who is going to allocate different weights based on context and the invest stratergy that user wants.
    In order to retreieve the context, use the search_tool and the create_embedding tool to search 'Tell me about the fininical siutuation of
    AAPL, MSFT, NVIDIA'. Combining this with the allocation stratergy that the user picks, redistribute the allocation percentages. Give exact figures 
    as to how this is to be used for each stock. You should also give a detailed explanation as to why you came up with these figures. Use the context
    to provide statistics in order to boost your reasoning. Make sure the reasoning is also returned in the final answwer.

    Query : {query}
    {answer}
    """
   
    examples = [
    {
        'query': 'Reallocte the portfolio for client 1 with a balanced strategy',
        'answer': ' The stock allocations for client_id 1 to 33,33,33'
    },

    {
        'query': 'Reallocte the portfolio for client 1 with a risk-based strategy',
        'answer': """
                    Allocating more to stocks with lower perceived risk. 
                    From my 10k data and online search, AAPL has strong financials
                    and minimal risk but NVIDIA has more risk. As a result, the stock
                    allocation for client_id 1 would have a high percentage for AAPL
                    and a low percentage for NVIDIA.
                    """
    },

    {
        'query': 'Reallocte the portfolio for client 1 with a return-based strategy',
        'answer': """
                    Allocating more to stocks with higher expected returns.
                    From my 10k data and online search, NVIDIA has higher expected growth rates and returns
                    whereas AAPL has a declining industry position and poor management quality. As a result, the stock
                    allocation for client_id 1 would have a high percentage for NVIDIA
                    and a low percentage for AAPL
                    """
    }
    ]
    
    example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template=template
    )

    
    #  2. Determine allocation stratergy based on query


    #  3. Read in the Financials from the 10k

    #  4. Get current information from online

    #  5. Apply the stratergy we determined on the information from steps 3 and 4!

    #  6. Translate this into a percentage allocation for each company and write to database

    prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Query: {input}",
    input_variables=["input"],
    )

    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)

    chain = prompt | llm

    ans = chain.invoke({'input' : query})

    return ans.content

    

tools = [Tool(name="search_tool", func=search_tool, description="Tool for performing online search operations"),
         Tool(name="create_embeddings", func=create_embeddings, description="Uses the content from the file path to create embeddings on the inputted data. Using the query provided, perform a similarity search on the query and return the output. If you can not provide a response,say I don't know rather than providing an incorrect response."),
         Tool(name='query_database', func=query_database, description= "This function queries the SQL database based on the query that the user provides to provide insight into the current client stock portfolio"),
         Tool(name='portfolio_allocation', func=portfolio_allocation, description='This function uses context from the vector database and online searches as well as the investment stratergy the user specifies to determine the stock allocation splits')]

In [498]:
template = """
You need to use the tools to find the appropriate action to take. 
Only use the query_database and portfolio_allocation tools when the term "portfolio" is mentioned in the query.

If further queries are presented alongside the term "portfilio", then use all tools available.
Otherwise, ignore these two tools and use the embeddings first
to see if you can get relevant responses from the similarity search from the user's query. If you do get relevant
responses, please return the responses from the similarity search. If you can't get relevent results from the embeddings,
use the seatch tool and get the reponse from there. If you still can't find an answer from an online search, please do not make up an
answer and return 'Sorry I can not answer your query' instead.


You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

If only the query_database tool is used, treat the tool as a function and return the full dictionary in the final answer.
For the values, ensure than every ticker(e.g. AAPL) is a key with the value pair being its stock allocation. If quieried for
multiple clients, data about ALL clients should be included. For example if allocations are asked for 'clients 1 and 2',
the allocations for BOTH of these clients should be returned and if the allocations are asked for 'all clients up to client 3',
the allocations for clients 1,2 and 3 should be returned and NOT just the allocations for client 3.

Similarly, if portfolio_allocation is used, return EXACTLY what is returned from the portfolio_allocation function.


Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

# Define the query and create the prompt
#query = 'return the values for client_id 1 in the sql database and also tell me the current apple stock price'
#query = 'Tell me the current apple stock price'
#query = 'Return me the stock allocation for client 1, client 2, and client 3'
#query = 'Return me the stock allocation for client 3 and client 7 and client 5'
#query = 'Return me the stock allocation for every client'
# query = 'give me all the stock allocations from all clients up to client 3'
#query = "The Company continues to develop new technologies to enhance existing products and services and to expand the range of its offerings through research and development RD licensing of intellectual property and acquisition of thirdparty businesses and technology"
#query = "Give me a sentence from the apple 10-k report"
#query = "what is the net sales of Macs in 2021"
#query =  "add a new client to the database with random stock allocations, provide the sql query to do so and execute it"
#query =  "set the stock allocations for client_id 1 to 5,5,90. provide the sql query to do so and execute it on the database" #remeber to pull numbers from another query into this framework




# query = 'give me all the stock allocations from all clients up to client 3'
query = 'give me all the stock allocations for all clients up to client 3'
# query = 'Reallocate the stocks for client 1 using a risk-based stratergy'

prompt = PromptTemplate(input_variables=['input','tools', 'agent_scratchpad', 'tool_names'], template=template)

# Initialize the language model
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)


# Create the agent using the language model and the toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Execute the agent with the given input
response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the query_database tool to retrieve the stock allocations for all clients up to client 3.

Action: query_database

Action Input: {'query': 'all clients up to client 3'}


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


stocks
Invoking: `sql_db_schema` with `{'table_names': 'stocks'}`



CREATE TABLE stocks (
	client_id INTEGER, 
	"AAPL" INTEGER, 
	"MSFT" INTEGER, 
	"NVIDIA" INTEGER, 
	PRIMARY KEY (client_id)
)

/*
3 rows from stocks table:
client_id	AAPL	MSFT	NVIDIA
1	5	5	90
2	20	40	40
3	10	10	80
*/
Invoking: `sql_db_query` with `{'query': 'SELECT client_id, "AAPL", "MSFT", "NVIDIA" FROM stocks WHERE client_id <= 3'}`
responded: Based on the schema of the `stocks` table, I can query the client_id and stock values for all clients up to client 3. Here is the query:

```sql
SELECT client_id, "AAPL", "MSFT", "NVIDIA"
FROM stocks
WHERE client_id <= 3
```

I will now execute this query to retrieve the

In [499]:
get_numbers('give me all the stock allocations for all clients up to client 3')





[1, 2, 3]

In [500]:
x = portfolio_allocation('Reallocte the portfolio for client 1 with a return.-based strategy')
print(x)

Based on a return-based strategy, the stock allocation for client 1 would be skewed towards NVIDIA due to its higher expected growth rates and returns compared to AAPL. Therefore, the allocation percentages would be higher for NVIDIA and lower for AAPL. Specific figures for the allocation could be 40% for NVIDIA and 20% for AAPL, with the remaining 40% allocated to MSFT.

This allocation is based on the analysis of the financial situation of the three stocks. NVIDIA has been showing strong growth potential and is expected to continue performing well in the future, making it a favorable choice for higher allocation. On the other hand, AAPL has been facing challenges in terms of industry position and management quality, leading to a lower allocation percentage.

By reallocating the portfolio in this manner, the client can potentially maximize their returns by capitalizing on the growth opportunities presented by NVIDIA while minimizing exposure to the risks associated with AAPL.
